--------------------------------------------------------------------------------

# Created by [Nikolaos G. Giakoumoglou](https://github.com/giakou4)

### Electrical and Computer Engineer

Mobile: +30 6906430995 | e-mail: [ngiakoumoglou@gmail.gr](mailto:ngiakoumoglou@gmail.gr) | e-mail: [ngiakoumoglou@hotmail.gr](mailto:ngiakoumoglou@hotmail.gr) |

-------------------------------------------------------------------------------

# 1 Install Detectron2 Dependencies

In [ ]:
%%capture
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
# opencv is pre-installed on colab

# install detectron2:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

import torch, torchvision
print('Torch version: ', torch.__version__, 'CUDA availiable: ', torch.cuda.is_available())
!gcc --version

In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

# 2 Detectron2 Dataset
We'll download our dataset from [Roboflow](https://roboflow.com/).

In [ ]:
!curl -L "https://app.roboflow.com/ds/WOqRf0wIHh?key=1N1AI973Z2" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/content/train/_annotations.coco.json", "/content/train")
register_coco_instances("my_dataset_val", {}, "/content/valid/_annotations.coco.json", "/content/valid")
register_coco_instances("my_dataset_test", {}, "/content/test/_annotations.coco.json", "/content/test")

## 2.1 Visualize Training Data

In [ ]:
train_metadata  = MetadataCatalog.get("my_dataset_train")
train_dataset_dict = DatasetCatalog.get("my_dataset_train")

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(train_dataset_dict, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata , scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

# 3 Train Detectron2 Detector


## 3.1 Model Zoo
Select from Model Zoo here: https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md#coco-object-detection-baselines

* All COCO models were trained on ```train2017``` and evaluated on ```val2017```.
* For Faster/Mask R-CNN, we provide baselines based on 3 different backbone combinations: ```FPN```, ```C4```, ```DC5```

In [ ]:
coco_detection = [
    'fast_rcnn_R_50_FPN_1x.yaml',
    'faster_rcnn_R_101_C4_3x.yaml',
    'faster_rcnn_R_101_DC5_3x.yaml',
    'faster_rcnn_R_101_FPN_3x.yaml',       # ResNet-101 FPN 3x
    'faster_rcnn_R_50_C4_1x.yaml',
    'faster_rcnn_R_50_C4_3x.yaml',
    'faster_rcnn_R_50_DC5_1x.yaml',
    'faster_rcnn_R_50_DC5_3x.yaml',
    'faster_rcnn_R_50_FPN_1x.yaml',        # (ResNet-50 FPN 1x)
    'faster_rcnn_R_50_FPN_3x.yaml',        # ResNet-50 FPN 3x
    'faster_rcnn_X_101_32x8d_FPN_3x.yaml', # ResNeXt-101 FPN 3x
    'fcos_R_50_FPN_1x.py',                 # (FCOS ResNet-50 FPN 1x)
    'retinanet_R_101_FPN_3x.yaml',         # (RetinaNet ResNet-101 FPN 3x)
    'retinanet_R_50_FPN_1x.py',
    'retinanet_R_50_FPN_1x.yaml',
    'retinanet_R_50_FPN_3x.yaml',          # (RetinaNet ResNet-50 FPN 3x)
    'rpn_R_50_C4_1x.yaml',
    'rpn_R_50_C4_1x.yaml',
]

coco_instance_segmentation = [
    'mask_rcnn_R_101_C4_3x.yaml',
    'mask_rcnn_R_101_DC5_3x.yaml',
    'mask_rcnn_R_101_FPN_3x.yaml',         # ResNet-101 FPN
    'mask_rcnn_R_50_C4_1x.py',
    'mask_rcnn_R_50_C4_1x.yaml',
    'mask_rcnn_R_50_C4_3x.yaml',
    'mask_rcnn_R_50_DC5_1x.yaml',
    'mask_rcnn_R_50_DC5_3x.yaml',
    'mask_rcnn_R_50_FPN_1x.py',
    'mask_rcnn_R_50_FPN_1x.yaml',
    'mask_rcnn_R_50_FPN_1x_giou.yaml',
    'mask_rcnn_R_50_FPN_3x.yaml',           # ResNet-50 FPN
    'mask_rcnn_X_101_32x8d_FPN_3x.yaml',    # ResNeXt-101 FPN
    'mask_rcnn_regnety_4gf_dds_fpn_1x.py',
]

## 3.2 Augmentation

In [ ]:
from detectron2.data import transforms as T

train_augmentations = [
    T.Resize((1024,1024)),
    T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
    T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
    T.RandomRotation(angle=[-0.15, 0.15]),
    T.RandomBrightness(0.5, 2),
    T.RandomContrast(0.5, 2),
    T.RandomSaturation(0.5, 2),
]

## 3.3 Mapper

In [ ]:
import copy
from detectron2.data import detection_utils as utils


def custom_mapper(dataset_dict):
    # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
    dataset_dict = copy.deepcopy(dataset_dict) 
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = train_augmentations
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

## 3.4 Trainer

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
from detectron2.data import build_detection_train_loader

class CustomTrainer(DefaultTrainer):
    """ 
    Custom Loader & Evaluation
    We are importing our own Trainer Module here
    to use the COCO validation evaluation during training.
    Otherwise no validation eval occurs.
    We are importing our own loader here
    to use custom augmentation.
    """

    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):

        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"

        return COCOEvaluator(dataset_name, cfg, False, output_folder)

## 3.5 CFG

In [ ]:
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 4
cfg.TEST.EVAL_PERIOD = 1000
# Input
cfg.INPUT.MIN_SIZE_TEST= 1024
cfg.INPUT.MAX_SIZE_TEST = 1024
cfg.INPUT.MIN_SIZE_TRAIN = 1024
cfg.INPUT.MAX_SIZE_TRAIN = 1024
# Model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4 # your number of classes + 1
# Optimizer - SGD
cfg.SOLVER.LR_SCHEDULER_NAME = 'WarmupMultiStepLR'
cfg.SOLVER.MAX_ITER = 1500
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.GAMMA = 0.05
cfg.SOLVER.STEPS = (100, 1500)
cfg.SOLVER.WARMUP_FACTOR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.CHECKPOINT_PERIOD = 1000

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

## 3.6 Train

In [ ]:
trainer = CustomTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

## 3.7 Tensorboard

In [ ]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

# 4 Evaluation

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.85

predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
inference_on_dataset(trainer.model, val_loader, evaluator)

# 5 Inference with Detectron2 Saved Weights



In [ ]:
%ls ./output/

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.DATASETS.TEST = ("my_dataset_test", )
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5

predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("my_dataset_test")

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob
import time

for imageName in glob.glob('/content/test/*jpg'):
  im = cv2.imread(imageName)
  start_time = time.time()
  outputs = predictor(im)
  print("Processed image in %s seconds." % (time.time() - start_time))
  v = Visualizer(im[:, :, ::-1], metadata=test_metadata,  scale=0.8)
  out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
  cv2_imshow(out.get_image()[:, :, ::-1])
